In [ ]:
result = dict()
for accessibility in accessibilities:
    result[accessibility] = opt.solve( household, current, potential, accessibility, budgets )

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pop as pop
from tqdm.notebook import tqdm
from pathlib import Path
import pyperclip

%matplotlib inline

# https://stackoverflow.com/questions/17687213/how-to-obtain-the-same-font-style-size-etc-in-matplotlib-output-as-in-latex
params = {'text.usetex' : True,
          'font.size'   : 11, # the article seems to be in 11pt, change accordingly
          'font.family' : 'lmodern',
          }

plt.rcParams.update(params)
default_size_inches = (3.54,3.54)
plt.rcParams['figure.figsize'] = default_size_inches

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import jg_util as jg

In [21]:
results_path = '../../../Results/Nepal/'
data_path = '../Results/' #results_path + 'Data/'
Path(results_path).mkdir(parents=True, exist_ok=True)

# Administrative Nepal

In [15]:
shapefile, data1, data2 = jg.GetAdministrativeNepal()

In [16]:
jg.DrawAdministrative(shapefile)

Error in callback <function _draw_all_if_interactive at 0x0000022035A5BA30> (for post_execute):


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 354x354 with 1 Axes>

In [17]:
pyperclip.copy( '\n'.join( pd.concat( (data1, data2), axis=1 ).style.hide(axis='index'). \
    to_latex().replace('.000000','').replace(' nan ',' ').split('\n')[2:-2] ) )

# Main data

In [20]:
accessibilities, current, potential = jg.GetAccessibilityData(data_path)


FileNotFoundError: [Errno 2] No such file or directory: '../Results/Travel Time/current_hospitals.pkl'

In [ ]:
household = jg.get_headcount(data_path)

In [ ]:
data1, data2 = jg.SplitPotentialSites(potential)

In [ ]:
with pd.option_context("max_colwidth", 1000):
    table_potential = pd.concat( (data1, data2), axis=1, ignore_index=True ).set_index(0,drop=True).style.to_latex()
table_potential = table_potential.replace('.000000','').replace(' nan ',' ')
pyperclip.copy('\n'.join(table_potential.split('\n')[3:-2]))

In [ ]:
adm_boundary = pd.read_excel('../Data/admboundary.xlsx',header=[0,1],index_col=0)
pyperclip.copy(adm_boundary.style.to_latex())

# Data in operation

In [ ]:
df_tests_lab = pd.read_excel('../Data/tests_per_lab.xlsx')
df_tests_lab['Date'] = pd.to_datetime(df_tests_lab['Date'])
df_tests_lab['ShortDate'] = df_tests_lab['Date'].dt.strftime('%m/%d')

In [ ]:
for_table = df_tests_lab[['ShortDate','Laboratory','Province Name']].sort_values(['ShortDate','Province Name','Laboratory']).drop_duplicates(subset='Laboratory',keep='first').reset_index(drop=True)
for_table

In [ ]:
for_table['Laboratory'] = [lab.replace('&','\&') for lab in for_table.Laboratory.values]
for_table[for_table.ShortDate=='05/01']

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html
with pd.option_context("max_colwidth", 1000):
    table = for_table.style.to_latex()
pyperclip.copy('\n'.join(table.split('\n')[2:-2]))

In [ ]:
past = df_tests_lab[['Date','Laboratory']]
past.columns = ['Date','L_NAME']

In [ ]:
past

In [ ]:
merged = dict()
for a in current.keys():
    merged[a] = pd.merge(past, current[a], on = 'L_NAME').sort_values('Date')

In [ ]:
cov_past = pd.DataFrame( index=sorted(list(set(past.Date))) )

In [ ]:
for a in merged.keys():
    cov_cols = [c for c in merged[a].columns if c.startswith('ID')]
    cov_cols.sort(key=jg.natural_keys,reverse=True)

    for c in cov_cols:
        cov_past[' '.join(c.split('_')[1:])] = [ jg.CoveragePerDay( merged[a], c, household, day ) for day in cov_past.index ]

In [ ]:
cov_past *= 100

In [ ]:
def format_name( name ):
    d,_,what = name.replace('km', ' km').replace( 'min', ' min' ).partition(' ')
    return '{:4d} {}'.format(int(d),what)

def range( df, column ):
    return( '{:2.0f} to {:2.0f}'.format( df[column].min(), df[column].max() ) )

cov_past.rename(columns = { c : range( cov_past, c ) + format_name(c) for c in cov_past.columns }, inplace = True)
cov_past


In [ ]:
jg.draw_lines( cov_past,
              y=cov_past.columns,
              x_title='Date',
              y_title='% population with access',
              file_name=results_path+'coverage_2021.pdf')

In [ ]:
jg.draw_lines( cov_past,
              y=[c for c in cov_past.columns if 'km' in c],
              x_title='Date',
              y_title='% population with access',
              file_name=results_path+'coverage_distance_2021.pdf')

In [ ]:
jg.draw_lines( cov_past,
              y=[c for c in cov_past.columns if 'driving' in c],
              x_title='Date',
              y_title='% population with access',
              file_name=results_path+'coverage_driving_2021.pdf')

In [ ]:
jg.draw_lines( cov_past,
              y=[c for c in cov_past.columns if 'walking' in c],
              x_title='Date',
              y_title='% population with access',
              file_name=results_path+'coverage_walking_2021.pdf')

In [ ]:
import glob

countPics = [ len( glob.glob( results_path+'Pics/{}/*.png'.format(a) ) ) for a in accessibilities ]

should_draw = min( countPics ) < 2*min( [ len(current[a]) for a in accessibilities ] )

In [ ]:
should_draw

In [ ]:
if should_draw:
    isogone = { 'Time' : jg.ShowIsoChrones, 'Distance' : jg.ShowIsoDistance }
    isopoints = { 'Time' : jg.ShowIsoChronesPoints, 'Distance' : jg.ShowIsoDistancePoints }
    for a in tqdm(accessibilities):
        Path(results_path+'Pics/'+a).mkdir(parents=True, exist_ok=True)
        for hospital,lat,lon in tqdm(current[a][['L_NAME','Latitude','Longitude']].values):
            jg.FoliumToPng( jg.FitAround( isogone[a](current[a],hospital), lat, lon, .25, .25 ), results_path+'Pics/'+a+'/'+hospital+'_gone', crop=None )
            jg.FoliumToPng( jg.FitAround( isopoints[a](current[a],pop,hospital), lat, lon, .35, .35 ), results_path+'Pics/'+a+'/'+hospital+'_points', crop=None )

In [ ]:
jg.draw_lines( df_tests_lab.groupby('Date')['Laboratory'].nunique().reset_index(),
              x='Date',y='Laboratory',
              x_title='Date',
              y_title='Number of Laboratories',
              file_name=results_path+'nof_labs_2021.pdf')

In [ ]:
sequence = merged['Time'][['Date','Hosp_ID']].set_index('Date',drop=True).drop_duplicates()

In [ ]:
story = {'cyan' : ('2021-01-01','2021-05-01'), 'lime' : ('2021-06-01','2021-09-01'), 'red' : ('2021-09-01','2021-12-01') }

In [ ]:
story = { c : sequence[ (pd.Timestamp(f) < sequence.index) & (sequence.index <= pd.Timestamp(t)) ].Hosp_ID.values for c,(f,t) in story.items() }

In [ ]:
top = current['Time'][['Hosp_ID','Latitude','Longitude','L_NAME']]
bottom = potential['Time'][['Cluster_ID','Latitude','Longitude','Name']]
bottom.columns = top.columns
locations = pd.concat([top,bottom],ignore_index=True).set_index('Hosp_ID',drop=False)

In [ ]:
jg.FoliumToPng( jg.ShowPoints( locations, story ), results_path+'labs', 5 )

In [ ]:
lat,lon = jg.Locate('Kathmandu')
jg.FoliumToPng( jg.FitAround(jg.ShowPoints( locations, story ), lat, lon, .1, .1 ), results_path+'labs_Kathmandu', 5 )

In [ ]:
import sys
sys.path.append('../optimization/')
import coverage as opt

In [ ]:
budgets = [5,10,25,50,100,250,500]

In [ ]:
jg.show_pareto(result, current, household, 'Time', width=800,height=350, file_name=results_path+'Pareto_pure_time.pdf' )

In [ ]:
jg.show_pareto(result, current, household, 'Distance', width=800,height=350, file_name=results_path+'Pareto_pure_distance.pdf' )

In [ ]:
(result['Time'][0].max()-result['Time'][0].min())*100

In [ ]:
enable_if_you_want_to_wait = False

In [ ]:
if enable_if_you_want_to_wait:
    for a in tqdm(accessibilities):
        for c in tqdm(result[a][0].columns):
            folder_with = results_path+'Pics/Coverage/'+a+'/'+c+'/with/'
            folder_without = results_path+'Pics/Coverage/'+a+'/'+c+'/without/'
            Path(folder_with).mkdir(parents=True, exist_ok=True)
            Path(folder_without).mkdir(parents=True, exist_ok=True)
            for b in tqdm(result[a][0].index):
                sol       = result[a][1].loc[b,c]
                nof       = len(sol)
                coverage  = result[a][0].loc[b,c]*100
                file_name = 'budget={:03d} nof={:03d} coverage={:4.1f}'.format(b,nof,coverage)
                map = jg.ShowAccessibility( current, potential, sol, pop, a, c, 
                                color_new='green', color_current='cyan', 
                                min_opacity_no_access = .005, min_opacity_access=.02 )
                jg.FoliumToPng(map,folder_with+file_name)
                map = jg.ShowAccessibility( current, potential, sol, pop, a, c, 
                                color_new=None, color_current=None, 
                                min_opacity_no_access = .005, min_opacity_access=.02 )
                jg.FoliumToPng(map,folder_without+file_name)

In [ ]:
if enable_if_you_want_to_wait:
    for a in tqdm(accessibilities):
        for c in tqdm(result[a][0].columns):
            folder_with = results_path+'Pics/Coverage/'+a+'/'+c+'/with/'
            folder_without = results_path+'Pics/Coverage/'+a+'/'+c+'/without/'
            Path(folder_with).mkdir(parents=True, exist_ok=True)
            Path(folder_without).mkdir(parents=True, exist_ok=True)
            
            covered = np.unique( np.concatenate( current[a]['ID_'+c] ) ).astype(np.uint) 
            percent_covered = household[covered].sum()/household.sum()    

            sol       = []
            b = nof   = len(sol)
            coverage  = percent_covered*100
            file_name = 'budget={:03d} nof={:03d} coverage={:4.1f}'.format(b,nof,coverage)
            map = jg.ShowAccessibility( current, potential, sol, pop, a, c, 
                            color_new='green', color_current='cyan', 
                            min_opacity_no_access = .005, min_opacity_access=.02 )
            jg.FoliumToPng(map,folder_with+file_name)
            map = jg.ShowAccessibility( current, potential, sol, pop, a, c, 
                            color_new=None, color_current=None, 
                            min_opacity_no_access = .005, min_opacity_access=.02 )
            jg.FoliumToPng(map,folder_without+file_name)

In [ ]:
def TeX_figure(accessibility,column,budgets,result,which):
  
  def format_name( name ):
    d,_,what = name.replace('km', ' km').replace( 'min', ' min' ).partition(' ')
    return '{:d} {}'.format(int(d),what.replace('_',' '))
  
  file_names = []
  captions   = []
  for budget in budgets:
    sol       = result[accessibility][1].loc[budget,column]
    nof       = len(sol)
    coverage  = result[accessibility][0].loc[budget,column]*100
    file_names.append( 'budget={:03d} nof={:03d} coverage={:4.1f}.png'.format(budget,nof,coverage) )
    captions.append( '{} out of {} opened covering {:.2f}\%'.format(nof,budget,coverage) )

  
  
  
  a, b, c, d, e, f = file_names
  A, B, C, D, E, F = captions
  G = f'{accessibility} accessibility within {format_name(column)} {which} pins'
  H = f'{accessibility}:{column}-{which}'

  s = \
fr'''\begin{{figure}}[H]
  \centering
  \begin{{tabular}}{{ccc}}
    \includegraphics[scale=.18]{{Figures/Coverage/{accessibility}/{column}/{which}/{a}}} & 
    \includegraphics[scale=.18]{{Figures/Coverage/{accessibility}/{column}/{which}/{b}}} & 
    \includegraphics[scale=.18]{{Figures/Coverage/{accessibility}/{column}/{which}/{c}}} \\ [.2\abovecaptionskip]
    {{\tiny {A}}} & 
    {{\tiny {B}}} &
    {{\tiny {C}}} \\
    [\abovecaptionskip]
    \includegraphics[scale=.18]{{Figures/Coverage/{accessibility}/{column}/{which}/{d}}} & 
    \includegraphics[scale=.18]{{Figures/Coverage/{accessibility}/{column}/{which}/{e}}} & 
    \includegraphics[scale=.18]{{Figures/Coverage/{accessibility}/{column}/{which}/{f}}} \\
    [.2\abovecaptionskip]
    {{\tiny {D}}} &
    {{\tiny {E}}} &
    {{\tiny {F}}} \\
  \end{{tabular}}
  \caption{{{G}}}\label{{fig:{H}}}
\end{{figure}}'''
  return s


In [ ]:
these_budgets = [10,25,50,100,250,500]

for a in tqdm(accessibilities):
    for c in tqdm(result[a][0].columns):
        for w in ['with','without']:
            with open(results_path+f'{a}-{c}-{w}.tex','w') as f:
                f.write(TeX_figure(a,c,these_budgets,result,w))

In [ ]:
result = dict()
for accessibility in accessibilities:
    result[accessibility] = opt.solve( household, current, potential, accessibility, budgets )

In [ ]:
open_result = dict()
for accessibility in accessibilities:
    open_result[accessibility] = opt.solve( household, current, potential, accessibility, budgets, opt.make_optimizer_using('cbc') )

In [ ]:
open_result

In [ ]:
new_current, new_potential = opt.GoBackInTime( df_tests_lab, current, potential, accessibilities, to_date = '05/01' )

In [ ]:
potential['Time'].shape, new_potential['Time'].shape, current['Time'].shape, new_current['Time'].shape

In [ ]:
new_result = dict()
for accessibility in accessibilities:
    new_result[accessibility] = opt.solve( household, new_current, new_potential, accessibility, [20] )

In [ ]:
current_coverage = opt.CurrentValues( current, household, accessibilities )

In [ ]:
increase = opt.Tree()
for a in accessibilities:
    for c in new_result[a][0].columns:
        increase[a][c] = (new_result[a][0].loc[20,c] - current_coverage[a][c])/current_coverage[a][c]

In [ ]:
increase

In [ ]:
'{:,}'.format(household.sum())

In [ ]:
what_if = pd.DataFrame.from_dict( { a : { c : '{:,.0f}'.format( household.sum()*v ) for c,v in values.items() } for a,values in increase.items() }, orient='index' )

In [ ]:
what_if

In [ ]:
import pyperclip

pyperclip.copy( what_if.style.to_latex().replace('nan','').replace('_',' ') )

In [ ]:
common = opt.Tree()
for a in accessibilities:
    current_labs = set(current[a].Hosp_ID.values)
    for c in new_result[a][1].columns:
        new_labs = set(new_result[a][1].loc[20,c])
        common[a][c] = current_labs.intersection(new_labs)

In [ ]:
common

In [ ]:
def format_name( name ):
        d,_,what = name.replace('km', ' km').replace( 'min', ' min' ).partition(' ')
        return '{:4d} {}'.format(int(d),what.replace('_',' '))

In [ ]:
collect = []
for a in common.keys():
    for c in common[a].keys():
        aux = for_table.loc[sorted(list(common[a][c]))][['Laboratory','Province Name']]
        with pd.option_context("max_colwidth", 1000):
            collect.append( aux.style.to_latex(column_format='|r|ll|',
                            position_float='centering',
                            position='H',
                            caption=f'Labs selected in common for {a.lower()} accessibility {format_name(c)}'
                            ) )

In [ ]:
pyperclip.copy( '\n\n'.join( collect ) )

In [ ]:
opt.ComputeCoverageFromSolutions( result, current, potential, household, accessibilities )

In [ ]:
pop_with_district = jg.get_pop_with_district()
rwi_district = jg.get_rwi_district()

In [ ]:
keys = []
for a in accessibilities:
    keys = keys + [ c for c in result[a][1].columns ]
keys

In [ ]:
def TeX_figure_RWI(scenario,budget,keys,caption,scale=.13,file_type='pdf'):
  
  file_names = []
  for key in keys:
    file_names.append( f'{key}_{budget}.{file_type}' )
    
  G = caption
  H = '-'.join(caption.split(' '))
  
  return \
fr'''\begin{{figure}}[H]
  \centering
  \begin{{tabular}}{{cc}}
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[0]}}} & 
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[1]}}} \\ [.2\abovecaptionskip]
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[2]}}} & 
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[3]}}} \\ [.2\abovecaptionskip]
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[4]}}} & 
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[5]}}} \\ [.2\abovecaptionskip]
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[6]}}} & 
    \includegraphics[scale={scale}]{{Figures/RWI/{scenario}/{file_names[7]}}} \\
  \end{{tabular}}
  \caption{{{G}}}\label{{fig:{H}}}
\end{{figure}}'''

In [ ]:
jg.GenerateRWIPlots( results_path, '2021', new_result, new_current, new_potential, pop_with_district, rwi_district, file_type='pdf' )

In [ ]:
jg.GenerateRWIPlots( results_path, 'UnbiasedFuture', result, current, potential, pop_with_district, rwi_district, file_type='pdf' )

In [ ]:
pyperclip.copy(TeX_figure_RWI('2021',0,keys,caption='May 2021',scale=.5))

In [ ]:
pyperclip.copy(TeX_figure_RWI('2021',20,keys,caption='November 2021',scale=.5))

In [ ]:
pyperclip.copy(TeX_figure_RWI('UnbiasedFuture',500,keys,caption='Full budget',scale=.5))

In [ ]:
household_rwi = jg.get_pop_rwi()
household = jg.get_headcount(data_path)

In [ ]:
result_rwi = dict()
for accessibility in accessibilities:
    result_rwi[accessibility] = opt.solve( household_rwi, current, potential, accessibility, budgets )

In [ ]:
coverage_rwi = opt.ComputeCoverageFromSolutions( result_rwi, current, potential, household, accessibilities )

In [ ]:
result_rwi

In [ ]:
aux_result = dict()
for a in accessibilities:
    aux_result[a] = (coverage_rwi[a],result_rwi[a][1])

In [ ]:
jg.show_pareto(aux_result, current, household, 'Distance', width=800,height=350, file_name=results_path+'Pareto_RWI_distance.pdf' )

In [ ]:
jg.show_pareto(aux_result, current, household, 'Time', width=800,height=350, file_name=results_path+'Pareto_RWI_time.pdf' )

In [ ]:
jg.GenerateRWIPlots( results_path, 'BiasedFuture', aux_result, current, potential, pop_with_district, rwi_district, file_type='pdf' )

In [ ]:
pyperclip.copy(TeX_figure_RWI('BiasedFuture',500,keys,caption='Full budget taking relative wealth index into account',scale=.5))